In [1]:
import numpy as np
import jax.numpy as jnp
from itertools import product

# 1. State Space

In [2]:
def create_state_space(options: dict[str, int]) -> tuple[np.ndarray, np.ndarray]:
    """Create state space object and indexer.

    We need to add the convention for the state space objects.

    Args:
        options (dict): Options dictionary.

    Returns:
        tuple:

        - state_space (np.ndarray): 2d array of shape (n_states, n_state_variables + 1)
            which serves as a collection of all possible states. By convention,
            the first column must contain the period and the last column the
            exogenous processes. Any other state variables are in between.
            E.g. if the two state variables are period and lagged choice and all choices
            are admissible in each period, the shape of the state space array is
            (n_periods * n_choices, 3).
        - map_state_to_index (np.ndarray): Indexer array that maps states to indexes.
            The shape of this object is quite complicated. For each state variable it
            has the number of possible states as rows, i.e.
            (n_poss_states_state_var_1, n_poss_states_state_var_2, ....).

    """
    n_periods = options["n_periods"]
    n_lagged_choices = options["n_discrete_choices"]

    n_mother_alive = 2
    n_father_alive = 2
    n_mother_age = ["n_discrete_choices"]
    n_father_age = ["n_discrete_choices"]
    n_mother_health = 3
    n_father_health = 3
    # n_dist_to_parents = ["n_discrete_choices"]
    n_has_sibling = 2

    shape = (
        n_periods,
        n_age,
        n_mother_alive,
        n_mother_age,
        n_mother_health,
        n_father_alive,
        n_father_age,
        n_father_health,
        n_has_sibling,
        n_lagged_choices,
        n_exog_states,
    )  # 15

    map_state_to_index = np.full(shape, -9999, dtype=np.int64)
    _state_space = []

    i = 0
    for period in range(n_periods):
        for lagged_choice in range(n_lagged_choices):
            for mother_alive in [0, 1]:
                for father_alive in [0, 1]:
                    for mother_age in range(n_mother_age):
                        for father_age in range(n_father_age):
                            for mother_health in range(mother_health):
                                for father_health in range(father_health):
                                    for has_sibling in [
                                        0,
                                        1,
                                    ]:
                                        for exog_process in range(n_exog_states):
                                            map_state_to_index[
                                                period,
                                                mother_alive,
                                                mother_age,
                                                mother_health,
                                                father_alive,
                                                father_age,
                                                father_health,
                                                dist_to_parents,
                                                sibling_present,
                                                #                                                                        lagged_choice,
                                                lagged_choice,
                                                exog_process,
                                            ] = i

                                            row = [
                                                period,
                                                mother_alive,
                                                mother_age,
                                                mother_health,
                                                father_alive,
                                                father_age,
                                                father_health,
                                                dist_to_parents,
                                                sibling_present,
                                                #                                                                        lagged_choice,
                                                lagged_choice,
                                                exog_process,
                                            ]
                                            _state_space.append(row)

                                            i += 1

    state_space = np.array(_state_space, dtype=np.int64)

    return state_space, map_state_to_index

In [3]:
def _create_state_space(options):
    shape = (
        n_periods,
        n_work_experience,
        n_years_since_retirement,
        n_caregiving_years,
        n_married,
        n_education_levels,
        n_married,
        n_mother_alive,
        n_mother_age,
        n_mother_health,
        n_father_alive,
        n_father_age,
        n_father_health,
        n_dist_to_parents,
        n_sibling_present,
        n_lagged_choices,
        n_exog_states,
    )  # 15

    map_state_to_index = np.full(shape, -9999, dtype=np.int64)
    _state_space = []

    for i, (
        period,
        mother_alive,
        mother_age,
        mother_health,
        father_alive,
        father_age,
        father_health,
        dist_to_parents,
        sibling_present,
        lagged_choice,
        exog_process,
    ) in enumerate(
        product(
            range(n_periods),
            [0, 1],
            [0, 1],
            [0, 1],
            range(n_mother_age),
            range(n_mother_health),
            [0, 1],
            range(n_father_age),
            range(n_father_health),
            range(n_dist_to_parents),
            [0, 1],
            range(n_lagged_choices),
            range(n_exog_states),
        ),
    ):
        map_state_to_index[
            period,
            mother_alive,
            mother_age,
            mother_health,
            father_alive,
            father_age,
            father_health,
            dist_to_parents,
            sibling_present,
            lagged_choice,
            exog_process,
        ] = i

        row = [
            period,
            mother_alive,
            mother_age,
            mother_health,
            father_alive,
            father_age,
            father_health,
            dist_to_parents,
            sibling_present,
            lagged_choice,
            exog_process,
        ]
        _state_space.append(row)

## Choice structure

### Labor choices (4)

- no work
- part-time
- full-time
- retirement (absorbing)

### Caregiving choices (6)

- light informal + no formal
- light informal + formal
- intensive informal + no_formal
- intensive informal + formal
- no informal + formal
- no informal + no formal (no care)

The outside care option (neither organize
formal care nor organize formal care once care demand arises) captures that siblings, the more healthy
parent or others organize or provide care to the parent. BFischer, p. 13

In [4]:
labor = ["retirement", "no_work", "part_time", "full_time"]
caregiving = [
    "no_informal_no_formal",
    "no_informal_formal",
    "light_informal_no_formal",
    "light_informal_formal",
    "intensive_informal_no_formal",
    "intensive_informal_formal",
]

combinations_dict = {}

for i, labor_element in enumerate(labor):
    for j, caregiving_element in enumerate(caregiving):
        key = i * len(caregiving) + j  # Generating unique keys
        value = [labor_element, caregiving_element]
        combinations_dict[key] = value

combinations_dict

{0: ['retirement', 'no_informal_no_formal'],
 1: ['retirement', 'no_informal_formal'],
 2: ['retirement', 'light_informal_no_formal'],
 3: ['retirement', 'light_informal_formal'],
 4: ['retirement', 'intensive_informal_no_formal'],
 5: ['retirement', 'intensive_informal_formal'],
 6: ['no_work', 'no_informal_no_formal'],
 7: ['no_work', 'no_informal_formal'],
 8: ['no_work', 'light_informal_no_formal'],
 9: ['no_work', 'light_informal_formal'],
 10: ['no_work', 'intensive_informal_no_formal'],
 11: ['no_work', 'intensive_informal_formal'],
 12: ['part_time', 'no_informal_no_formal'],
 13: ['part_time', 'no_informal_formal'],
 14: ['part_time', 'light_informal_no_formal'],
 15: ['part_time', 'light_informal_formal'],
 16: ['part_time', 'intensive_informal_no_formal'],
 17: ['part_time', 'intensive_informal_formal'],
 18: ['full_time', 'no_informal_no_formal'],
 19: ['full_time', 'no_informal_formal'],
 20: ['full_time', 'light_informal_no_formal'],
 21: ['full_time', 'light_informal_for

### 24 choices in total

### ==> structure !!! keep numbers/mapping in mind !!!

In [5]:
formal = [
    combinations_dict[1],
    combinations_dict[3],
    combinations_dict[5],
    combinations_dict[7],
    combinations_dict[9],
    combinations_dict[11],
    combinations_dict[13],
    combinations_dict[15],
    combinations_dict[17],
    combinations_dict[19],
    combinations_dict[21],
    combinations_dict[23],
]

light_informal = [
    combinations_dict[2],
    combinations_dict[3],
    combinations_dict[8],
    combinations_dict[9],
    combinations_dict[14],
    combinations_dict[15],
    combinations_dict[20],
    combinations_dict[21],
]

intensive_informal = [
    combinations_dict[4],
    combinations_dict[5],
    combinations_dict[10],
    combinations_dict[11],
    combinations_dict[16],
    combinations_dict[17],
    combinations_dict[22],
    combinations_dict[23],
]

In [6]:
light_informal

[['retirement', 'light_informal_no_formal'],
 ['retirement', 'light_informal_formal'],
 ['no_work', 'light_informal_no_formal'],
 ['no_work', 'light_informal_formal'],
 ['part_time', 'light_informal_no_formal'],
 ['part_time', 'light_informal_formal'],
 ['full_time', 'light_informal_no_formal'],
 ['full_time', 'light_informal_formal']]

In [7]:
intensive_informal

[['retirement', 'intensive_informal_no_formal'],
 ['retirement', 'intensive_informal_formal'],
 ['no_work', 'intensive_informal_no_formal'],
 ['no_work', 'intensive_informal_formal'],
 ['part_time', 'intensive_informal_no_formal'],
 ['part_time', 'intensive_informal_formal'],
 ['full_time', 'intensive_informal_no_formal'],
 ['full_time', 'intensive_informal_formal']]

In [8]:
formal

[['retirement', 'no_informal_formal'],
 ['retirement', 'light_informal_formal'],
 ['retirement', 'intensive_informal_formal'],
 ['no_work', 'no_informal_formal'],
 ['no_work', 'light_informal_formal'],
 ['no_work', 'intensive_informal_formal'],
 ['part_time', 'no_informal_formal'],
 ['part_time', 'light_informal_formal'],
 ['part_time', 'intensive_informal_formal'],
 ['full_time', 'no_informal_formal'],
 ['full_time', 'light_informal_formal'],
 ['full_time', 'intensive_informal_formal']]

## 2. Utility

In [9]:
def utility_func_crra(consumption: jnp.array, choice: int, params: dict) -> jnp.array:
    """Computes the agent's current utility based on a CRRA utility function.

    Args:
        consumption (jnp.array): Level of the agent's consumption.
            Array of shape (i) (n_quad_stochastic * n_grid_wealth,)
            when called by :func:`~dcgm.call_egm_step.map_exog_to_endog_grid`
            and :func:`~dcgm.call_egm_step.get_next_period_value`, or
            (ii) of shape (n_grid_wealth,) when called by
            :func:`~dcgm.call_egm_step.get_current_period_value`.
        choice (int): Choice of the agent, e.g. 0 = "retirement", 1 = "working".
        params (dict): Dictionary containing model parameters.
            Relevant here is the CRRA coefficient theta.

    Returns:
        utility (jnp.array): Agent's utility . Array of shape
            (n_quad_stochastic * n_grid_wealth,) or (n_grid_wealth,).

    """
    theta = params["theta"]

    working_hours = 8
    informal_caregiving_hours = 0
    leisure_hours = 24 - working_hours - informal_caregiving_hours

    age = period  # + 50 # 55

    care_demand = 0
    unobserved_type = 0

    formal_care = choice % 2 == 1  # uneven numbers mark formal care
    light_informal_care = choice in [2, 3, 8, 9, 14, 15, 20, 21]
    intensive_informal_care = choice in [4, 5, 10, 11, 16, 17, 22, 23]

    utility_consumption = (consumption ** (1 - theta) - 1) / (1 - theta)

    utility = (
        utility_consumption
        - (choice >= 12) * params["theta"]  # choice: part-time or full-time
        ## utility from leisure
        # type A
        + (unobserved_type == 0)
        * (params["utility_type_A"] + params["utility_age"] * age)
        * np.log(leisure_hours)
        + (unobserved_type == 1)
        # type B
        * (params["utility_type_B"] + params["utility_age"] * age)
        * np.log(leisure_hours)
        + care_demand * ()
        ## utility from caregiving
        # type A
        + (unobserved_type == 0)
        * (
            params["utility_light_informal_type_A"] * (choice == 0)
            + params["utility_intensive_informal_type_A"] * (choice == 0)
            + params["utility_formal_type_A"] * (choice == 0)
            + params["utility_informal_and_formal_type_A"] * (choice == 0)
        )
        # type B
        + (unobserved_type == 1)
        * (
            params["utility_light_informal_type_B"] * (choice == 0)
            + params["utility_intensive_informal_type_B"] * (choice == 0)
            + params["utility_formal_type_B"] * (choice == 0)
            + params["utility_informal_and_formal_type_B"] * (choice == 0)
        )
    )

    return utility

- Age is a proxy for health impacting the taste for free-time
- Formal care, retirement and unemployment do not reduce leisure time.

## Write tests on this!!

In [10]:
def test_choice(choice):
    formal_care = choice % 2 == 1  # uneven numbers mark formal care
    light_informal_care = choice in [2, 3, 8, 9, 14, 15, 20, 21]
    intensive_informal_care = choice in [4, 5, 10, 11, 16, 17, 22, 23]

    # light and intensive can never be true at the same time: axis (1, 2)

    return formal_care, light_informal_care, intensive_informal_care

In [11]:
for choice in range(24):
    print(test_choice(choice))

(False, False, False)
(True, False, False)
(False, True, False)
(True, True, False)
(False, False, True)
(True, False, True)
(False, False, False)
(True, False, False)
(False, True, False)
(True, True, False)
(False, False, True)
(True, False, True)
(False, False, False)
(True, False, False)
(False, True, False)
(True, True, False)
(False, False, True)
(True, False, True)
(False, False, False)
(True, False, False)
(False, True, False)
(True, True, False)
(False, False, True)
(True, False, True)


In [12]:
combinations_dict

{0: ['retirement', 'no_informal_no_formal'],
 1: ['retirement', 'no_informal_formal'],
 2: ['retirement', 'light_informal_no_formal'],
 3: ['retirement', 'light_informal_formal'],
 4: ['retirement', 'intensive_informal_no_formal'],
 5: ['retirement', 'intensive_informal_formal'],
 6: ['no_work', 'no_informal_no_formal'],
 7: ['no_work', 'no_informal_formal'],
 8: ['no_work', 'light_informal_no_formal'],
 9: ['no_work', 'light_informal_formal'],
 10: ['no_work', 'intensive_informal_no_formal'],
 11: ['no_work', 'intensive_informal_formal'],
 12: ['part_time', 'no_informal_no_formal'],
 13: ['part_time', 'no_informal_formal'],
 14: ['part_time', 'light_informal_no_formal'],
 15: ['part_time', 'light_informal_formal'],
 16: ['part_time', 'intensive_informal_no_formal'],
 17: ['part_time', 'intensive_informal_formal'],
 18: ['full_time', 'no_informal_no_formal'],
 19: ['full_time', 'no_informal_formal'],
 20: ['full_time', 'light_informal_no_formal'],
 21: ['full_time', 'light_informal_for

## 3. Budget Constraint

In [13]:
def budget(lagged_resources, lagged_consumption, lagged_choice, wage, health, params):
    interest_factor = 1 + params["interest_rate"]

    light_informal_care = [2, 3, 8, 9, 14, 15, 20, 21]
    intensive_informal_care = [4, 5, 10, 11, 16, 17, 22, 23]

    health_costs = params["ltc_cost"]

    resources = (
        interest_factor * (lagged_resources - lagged_consumption)  # = savings (:
        + wage * (lagged_choice >= 12) * working_hours
        + spousal_income * lagged_married
        + retirement_benefits * (lagged_choice <= 5)
        + unemployment_benefits * (6 <= lagged_choice < 12)
        + cash_benefits_informal_caregiving
        + inheritance  # depends on caregiving? See maybe Korfhage
        * (lagged_choice in [light_informal_care + intensive_informal_care])
        # costs in current period
        - (choice >= 12) * (social_security_contributions + taxes)  # only if working?
        - (choice % 2 == 1) * formal_care_costs
    ).clip(
        min=0.5,
    )  # why clip?

    return resources

In [14]:
light_informal_care = [2, 3, 8, 9, 14, 15, 20, 21]
intensive_informal_care = [4, 5, 10, 11, 16, 17, 22, 23]

In [15]:
light_informal_care + intensive_informal_care

[2, 3, 8, 9, 14, 15, 20, 21, 4, 5, 10, 11, 16, 17, 22, 23]

In [16]:
lagged_choice = 7

In [17]:
5 <= lagged_choice < 12

True

In [18]:
# non-labor income Skira
# capture inheritances!!
# Skira: job dynamics, part-time penalty
# --> wage offer

In [19]:
def wage(state, param):
    wage = 0

    return wage

In the model, caregiving decisions are not motivated by inheritances or inter-vivos transfers. Most recent studies
do not support the bequest motive (Checkovich and Stern, 2002; Norton and Van Houtven, 2006; Brown, 2007). The
evidence on inter-vivos transfers is mixed. McGarry and Schoeni (1997) and Brown (2006) find parents do not transfer
significantly more to their caregiving children than their noncaregiving children on average, whereas Norton and Van
Houtven (2006) find caregiving children are 11–16 percentage points more likely to receive an inter-vivos transfer. (Skira, p 68)

In [20]:
# see korfhage for expected value of caregiving reimbursment, inheritance etc.

In [21]:
def inheritance(state):
    inheritance = 0
    return inheritance

## Transition probabilities

# 2. Simulation

## 2.1. Choice probabilities

## 2.2